FaceMask Detection on edgetpu using mobiledet ssdlite with kaggle dataset.

In [3]:
# Import tensorflow 1.x and install tf_slim
%tensorflow_version 1.x
!pip install tf_slim
!pip show tensorflow

Name: tensorflow
Version: 1.15.2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /tensorflow-1.15.2/python3.6
Requires: keras-preprocessing, absl-py, six, tensorflow-estimator, keras-applications, astor, protobuf, google-pasta, opt-einsum, tensorboard, grpcio, gast, wheel, termcolor, wrapt, numpy
Required-by: stable-baselines, magenta, fancyimpute


In [4]:
# Install protobuf-compiler and object detection API.
!apt-get install protobuf-compiler imagemagick
!git clone https://github.com/tensorflow/models.git

import os
os.environ['PYTHONPATH'] += ':/content/models/research/'
os.environ['PYTHONPATH'] += ':/content/models/research/slim/'
os.environ['PYTHONPATH'] += ':/content/models/research/object_detection/utils/'
os.environ['PYTHONPATH'] += ':/content/models/research/object_detection'

%cd models/research
# Compile all the protobuf dependencies.
!protoc object_detection/protos/*.proto --python_out=.
# Set up and install the object detection API.
!cp object_detection/packages/tf1/setup.py .
!python -m pip install .
# Run a test to make sure setup is correct.
!python object_detection/builders/model_builder_test.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
imagemagick is already the newest version (8:6.9.7.4+dfsg-16ubuntu6.8).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
Cloning into 'models'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 47579 (delta 2), reused 8 (delta 0), pack-reused 47569
Receiving objects: 100% (47579/47579), 551.84 MiB | 19.98 MiB/s, done.
Resolving deltas: 100% (32773/32773), done.
/content/models/research/models/research
Processing /content/models/research/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1599232 sha256=1d161f8a50381dc249b6318cb9eb5326547463a01a856b26b0f9af5abb995e87
  Stored in directory: /tmp/pip-ephem-wheel-cache-m0saji21/wheels/9d/3f/b5/750377a9d856e59653

In [5]:
# Download the Mask Dataset from MakeML: https://makeml.app/datasets/mask
# License: Public Domain Mark
!mkdir /content/dataset
%cd /content/dataset
!wget https://arcraftimages.s3-accelerate.amazonaws.com/Datasets/Mask/MaskPascalVOC.zip
!unzip MaskPascalVOC.zip
!rm -r __* # remove MacOS cache

/content/dataset
--2020-11-23 14:18:14--  https://arcraftimages.s3-accelerate.amazonaws.com/Datasets/Mask/MaskPascalVOC.zip
Resolving arcraftimages.s3-accelerate.amazonaws.com (arcraftimages.s3-accelerate.amazonaws.com)... 13.32.141.175
Connecting to arcraftimages.s3-accelerate.amazonaws.com (arcraftimages.s3-accelerate.amazonaws.com)|13.32.141.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 417606530 (398M) [application/zip]
Saving to: ‘MaskPascalVOC.zip’

MaskPascalVOC.zip   100%[===================>] 398.26M  52.6MB/s    in 8.5s    

2020-11-23 14:18:23 (47.1 MB/s) - ‘MaskPascalVOC.zip’ saved [417606530/417606530]

Archive:  MaskPascalVOC.zip
   creating: annotations/
  inflating: __MACOSX/._annotations  
  inflating: annotations/maksssksksss299.xml  
  inflating: __MACOSX/annotations/._maksssksksss299.xml  
  inflating: annotations/maksssksksss528.xml  
  inflating: __MACOSX/annotations/._maksssksksss528.xml  
  inflating: annotations/maksssksksss272.x

In [6]:
# Lets create a mask_label_map.pbtxt file
%%file mask_label_map.pbtxt
item {
  id: 1
  name: 'with_mask'
}

item {
  id: 2
  name: 'without_mask'
}

item {
  id: 3
  name: 'mask_weared_incorrect'
}

Writing mask_label_map.pbtxt


In [7]:
# Here we plit the images and labels to test vs train
!mkdir /content/dataset/test
!mkdir /content/dataset/train
%cd /content/dataset/images
# Move first 25 images to test
!for i in {0..100}; do mv maksssksksss$i.png /content/dataset/test; done
# Move the rest to train
!mv mak* /content/dataset/train

# Same thing for annotations
%cd /content/dataset/annotations
# Move first 25 annotations to test
!for i in {0..100}; do mv maksssksksss$i.xml /content/dataset/test; done
# Move the rest to train
!mv mak* /content/dataset/train

/content/dataset/images
/content/dataset/annotations


In [8]:
# Now let's download our ssdlite mobiledet pretrained model from tensorflow's model zoo.
!mkdir /content/pretrained_model
%cd /content/pretrained_model
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz
!tar xvf ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz

/content/pretrained_model
--2020-11-23 14:18:31--  http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.133.128, 2a00:1450:400c:c02::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.133.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156413934 (149M) [application/x-tar]
Saving to: ‘ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz’

ssdlite_mobiledet_e 100%[===================>] 149.17M  31.6MB/s    in 4.7s    

2020-11-23 14:18:37 (31.6 MB/s) - ‘ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz’ saved [156413934/156413934]

ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/model.ckpt-400000.data-00000-of-00001
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/model.ckpt-400000

In [9]:
# This script will turn our xml files from each image to a single label file.
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path):
  xml_list = []
  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      value = (root.find('filename').text,
        int(root.find('size')[0].text),
        int(root.find('size')[1].text),
        member.find('name').text,
        int(member.find('bndbox')[0].text),
        int(member.find('bndbox')[1].text),
        int(member.find('bndbox')[2].text),
        int(member.find('bndbox')[3].text)
        )
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name)
  return xml_df


def get_labels_from_image_dir():
  for name in ['test', 'train']:
    full_dir = '/content/dataset/' + name + '/'
    label_file = '/content/dataset/' + name + '_labels.csv'
    image_path = os.path.join(os.getcwd(), full_dir)
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv(label_file, index=None)

get_labels_from_image_dir()

print('Training labels:')
!head /content/dataset/train_labels.csv
print('Testing labels:')
!head /content/dataset/test_labels.csv

Training labels:
filename,width,height,class,xmin,ymin,xmax,ymax
maksssksksss493.png,301,400,with_mask,62,165,182,315
maksssksksss264.png,400,267,with_mask,21,54,90,119
maksssksksss264.png,400,267,with_mask,165,18,194,47
maksssksksss264.png,400,267,with_mask,145,89,195,139
maksssksksss264.png,400,267,with_mask,110,218,233,267
maksssksksss264.png,400,267,with_mask,275,88,320,136
maksssksksss264.png,400,267,with_mask,244,18,274,48
maksssksksss264.png,400,267,with_mask,338,12,374,48
maksssksksss365.png,400,240,with_mask,129,118,171,167
Testing labels:
filename,width,height,class,xmin,ymin,xmax,ymax
maksssksksss88.png,301,400,with_mask,42,176,151,304
maksssksksss9.png,267,400,mask_weared_incorrect,148,75,201,133
maksssksksss9.png,267,400,without_mask,27,78,56,106
maksssksksss79.png,400,400,with_mask,60,121,87,148
maksssksksss79.png,400,400,with_mask,183,117,208,146
maksssksksss79.png,400,400,with_mask,239,165,256,183
maksssksksss79.png,400,400,with_mask,282,149,300,169
maksssksksss79.png,4

In [10]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict


def class_text_to_int(row_label):
    if row_label == 'with_mask':
        return 1
    elif row_label == 'without_mask':
        return 2
    elif row_label == 'mask_weared_incorrect':
        return 3
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = [] 
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main():
  for name in ['test', 'train']:
    writer = tf.python_io.TFRecordWriter('/content/dataset/' + name + '.record')
    path = os.path.join('/content/dataset/'+ name)
    examples = pd.read_csv('/content/dataset/' + name + '_labels.csv')
    grouped = split(examples, 'filename')
    for group in grouped:
      tf_example = create_tf_example(group, path)
      writer.write(tf_example.SerializeToString())
    writer.close()

main()

In [14]:
# Edit Pipeline config to load in our new tfrecord that we just created.
import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

pipeline = pipeline_pb2.TrainEvalPipelineConfig()                                                                                                                                                                                                          
config_path = '/content/models/research/object_detection/samples/configs/ssdlite_mobiledet_edgetpu_320x320_coco_sync_4x4.config'
with tf.gfile.GFile(config_path, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline)

pipeline.train_input_reader.tf_record_input_reader.input_path[:] = ['/content/dataset/train.record'] 
pipeline.train_input_reader.label_map_path = '/content/dataset/mask_label_map.pbtxt'
pipeline.eval_input_reader[0].tf_record_input_reader.input_path[:] = ['/content/dataset/test.record'] 
pipeline.eval_input_reader[0].label_map_path = '/content/dataset/mask_label_map.pbtxt'
pipeline.train_config.fine_tune_checkpoint = '/content/pretrained_model/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/uint8/model.ckpt'
pipeline.model.ssd.num_classes = 3
pipeline.model.ssd.image_resizer.fixed_shape_resizer.height = 320
pipeline.model.ssd.image_resizer.fixed_shape_resizer.width = 320

pipeline.train_config.batch_size = 64
pipeline.train_config.num_steps = 2000
pipeline.eval_config.num_examples = 100
# Quantization Aware training
pipeline.graph_rewriter.quantization.delay = 0
pipeline.graph_rewriter.quantization.weight_bits = 8
pipeline.graph_rewriter.quantization.activation_bits = 8
# Shuffle testing data
pipeline.eval_input_reader[0].shuffle = True

config_text = text_format.MessageToString(pipeline)                                                                                                                                                                                                        
with tf.gfile.Open(config_path, "wb") as f:                                                                                                                                                                                                                       
    f.write(config_text)

# This is out config after modifying.
!cat /content/models/research/object_detection/samples/configs/ssdlite_mobiledet_edgetpu_320x320_coco_sync_4x4.config

Mon Nov 23 14:23:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
# Before we start training, let's start tensorboard so we can track the progress.
# More info on tensorflow can be found here: https://www.tensorflow.org/tutorials
%cd /content
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

/content
--2020-11-23 14:23:47--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.20.238.4, 107.21.11.91, 54.175.121.11, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.20.238.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  14.3MB/s    in 0.9s    

2020-11-23 14:23:49 (14.3 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [18]:
%cd /content
# Starts tensorboard.
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format('/content/train')
)
get_ipython().system_raw('./ngrok http 6006 &')

print('Click on link below to track progress:')
import time
time.sleep(1)
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

/content
Click on link below to track progress:
https://85a0a74d97a0.ngrok.io


In [ ]:
# Change into the research directory and begin training.
%cd /content/models/research/
!python3 object_detection/model_main.py \
    --logtostderr=true \
    --model_dir=/content/train \
    --pipeline_config_path=/content/models/research/object_detection/samples/configs/ssdlite_mobiledet_edgetpu_320x320_coco_sync_4x4.config

/content/models/research
W1123 14:25:24.151241 140282653464448 model_lib.py:793] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: None
I1123 14:25:24.151627 140282653464448 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1123 14:25:24.151814 140282653464448 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I1123 14:25:24.151953 140282653464448 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1123 14:25:24.152095 140282653464448 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1123 14:25:24.152256 140282653464448 model_lib.py:809] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: u

In [ ]:
# Make inference graph.
!python3 /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/models/research/object_detection/samples/configs/ssdlite_mobiledet_edgetpu_320x320_coco_sync_4x4.config \
    --output_directory=/content/fine_tuned_model \
    --trained_checkpoint_prefix=/content/train/model.ckpt-2000

python3: can't open file '/content/models/research/object_detection/export_inference_graph.py': [Errno 2] No such file or directory


In [ ]:
# Do a Quick Evaluation on the graph model/
import pathlib
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
# This is needed to display the images.
%matplotlib inline

PATH_TO_CKPT = '/content/fine_tuned_model/frozen_inference_graph.pb'
PATH_TO_LABELS = '/content/dataset/mask_label_map.pbtxt'
NUM_CLASSES = 3

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(category_index)


PATH_TO_TEST_IMAGES_DIR = '/content/dataset/test'
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, 'maksssksksss{}.png'.format(i)) for i in range(0, 10)]

def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      # output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

for image_path in TEST_IMAGE_PATHS:
  print('Evaluating:', image_path)
  image = Image.open(image_path)
  img_width, img_height = image.size
  # Removing the Alpha Channel
  if image.mode == 'RGBA':
    image.load()
    removed_A = Image.new('RGB', image.size, (255,255,255))
    removed_A.paste(image, mask=image.split()[3])
    data = np.asarray(removed_A.getdata())
  else:  
    data = np.asarray(image.getdata())
  image_np = data.reshape((img_height, img_width, 3)).astype(np.uint8)
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=5)
  plt.figure(figsize=(12, 8))
  plt.imshow(image_np)

In [ ]:
# Export tflite ssd graph
%cd /content/models/research
!mkdir /content/output_model
!python3 object_detection/export_tflite_ssd_graph.py \
  --pipeline_config_path=/content/models/research/object_detection/samples/configs/ssdlite_mobiledet_edgetpu_320x320_coco_sync_4x4.config \
  --trained_checkpoint_prefix=/content/train/model.ckpt-2000 \
  --output_directory=/content/output_model \
  --add_postprocessing_op=true


In [ ]:
# Now we can convert this custom trained model to a CPU tflite model :)
!tflite_convert \
  --output_file="/content/output_model/ssdlite_mobiledet_mask.tflite" \
  --graph_def_file="/content/output_model/tflite_graph.pb" \
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays="normalized_input_image_tensor" \
  --output_arrays="TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3" \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,320,320,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops

In [ ]:
# Now we install tflite_runtime package to evaluate the model.
!pip3 install https://github.com/google-coral/pycoral/releases/download/release-frogfish/tflite_runtime-2.5.0-cp36-cp36m-linux_x86_64.whl  

In [ ]:
# Now we do evaluation on the tflite model.
from tflite_runtime.interpreter import Interpreter
from tflite_runtime.interpreter import load_delegate
from PIL import Image
from PIL import ImageDraw
%matplotlib inline

PATH_TO_MODEL = '/content/output_model/ssdlite_mobiledet_mask.tflite'
labels = {0:'mask', 1:'no_mask', 2:'incorrect'}
interpreter = Interpreter(PATH_TO_MODEL)

interpreter.allocate_tensors()
interpreter.invoke() # warmup
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
width = input_details[0]['shape'][2]
height = input_details[0]['shape'][1]

PATH_TO_TEST_IMAGES_DIR = '/content/dataset/test'
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, 'maksssksksss{}.png'.format(i)) for i in range(0, 100)]

def run_inference_for_single_image(image, interpreter):
  interpreter.set_tensor(input_details[0]['index'], image)
  interpreter.invoke()
  boxes = interpreter.get_tensor(output_details[0]['index'])[0]
  classes = interpreter.get_tensor(output_details[1]['index'])[0]
  scores = interpreter.get_tensor(output_details[2]['index'])[0]
  
  return boxes, classes, scores

colors = {0:(128, 255, 102), 1:(102, 255, 255), 2:(232, 123, 212)}
for image_path in TEST_IMAGE_PATHS:
  print('Evaluating:', image_path)
  image = Image.open(image_path)
  image_width, image_height = image.size
  draw = ImageDraw.Draw(image)
  # Removing the Alpha Channel
  if image.mode == 'RGBA':
    image.load()
    removed_A = Image.new('RGB', image.size, (255,255,255))
    removed_A.paste(image, mask=image.split()[3])
    resized_image = removed_A.resize((width, height))
  else:  
    resized_image = image.resize((width, height))
  np_image = np.asarray(resized_image)
  image_np_expanded = np.expand_dims(np_image, axis=0)
  # Actual detection.
  boxes, classes, scores = run_inference_for_single_image(image_np_expanded, interpreter)
  # Visualization of the results of a detection.
  for i in range(len(boxes)):
    if scores[i] > 0.5:
      ymin = int(max(1, (boxes[i][0] * image_height)))
      xmin = int(max(1, (boxes[i][1] * image_width)))
      ymax = int(min(image_height, (boxes[i][2] * image_height)))
      xmax = int(min(image_width, (boxes[i][3] * image_width)))
      draw.rectangle((xmin, ymin, xmax, ymax), width=3, outline=colors[int(classes[i])])
      draw.rectangle((xmin, ymin, xmax, ymin-10), fill=colors[int(classes[i])])
      text = labels[int(classes[i])] + ' ' + str(scores[i]*100) + '%'
      draw.text((xmin+2, ymin-10), text, fill=(0,0,0), width=2)
  display(image)

In [ ]:
# Install the edgetpu compiler.
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update
!sudo apt-get install edgetpu-compiler

In [ ]:
# Compile our model and make a tarball of the finished trained model.
%cd /content/output_model
!edgetpu_compiler -s ssdlite_mobiledet_mask.tflite
%cd /content/
# Copy the checkpoints, inference graph, pipeline config, and the tflite models
!cp -r /content/train/model.ckpt-2000* /content/output_model
!cp -r /content/inference_graph/* /content/output_model
!tar cvf facessd_mask.tar.gz output_model

In [ ]:
# Download model and you're done!
from google.colab import files
files.download('/content/facessd_mask.tar.gz')